# Using Word2Vec for Topic Modeling

The idea of using an $N$ dimensional feature vector for unsupervised learning a powerful concept.  If we can take a body of words and consider them as a vector, we can begin to interpret words and sentences as points in $N$ dimensional space, and use traditional methods, including the K Means algorithm, to assign labels to unstructured data.

In fact, the most complex algorithm in this notebook is *not* the K Means algorithm, but rather the preparation of the words to be used by the algorithm, which is the Word2Vec algorithm.  Word2Vec is a member of the class of algorithms that carry out a *word embedding*, which simply means that we take a collection of words and return a feature vector.  The feature vector is always of the same dimension, even though the collection of words may not be.   

An amazing property of word2vec is that vector embeddings can often result in interesting geometric interpretations, including additive properties like 

$$\text{w2v}(king) - \text{w2v}(queen) = \text{w2v}(man) - \text{w2v}(woman)$$, 

which suggests that an article, or body of text, can be a superposition of word vectors whose position in feature space relative to other articles can indicate their similarity.  This is a very helpful insight.

Let's take a look at this algorithm in action.  First, let's import some libraries:

In [1]:
// command line operations
import scala.sys.process._

// Spark Classes
import scala.collection.mutable
//import org.apache.spark.mllib.clustering.LDA
//import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.Word2Vec
import org.apache.spark.mllib.linalg.{Vector, Vectors}

// .tar and .gz utilities
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.spark.input.PortableDataStream
import scala.util.Try
import java.nio.charset._

// rounding function for time:
def round(x: Long) ={Math.round(x/1e6)/1e3}

# Loading the Corpus

We will download the yahoo 20 newsgroups dataset if you haven't already done so. This dataset contains 20,000 news stories that have been manually categorized into directories.  

In [2]:
import sys.process._
val listing = "ls 20_newsgroups"!! ;

if (listing.contains("20_newsgroups") == false ) {
    println("downloading dataset")
    "wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz"!;
    "tar -xvf 20news-19997.tar.gz"!;
    "rm 20news-19997*"!;
}
println("directories in 20 newsgroup dataset")


--2018-11-25 20:15:31--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
downloading dataset
Resolving proxy1.fyre.ibm.com (proxy1.fyre.ibm.com)... 172.16.200.60, 172.16.200.61
Connecting to proxy1.fyre.ibm.com (proxy1.fyre.ibm.com)|172.16.200.60|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz.4’

     0K .......... .......... .......... .......... ..........  0% 58.5M 0s
    50K .......... .......... .......... .......... ..........  0% 80.5M 0s
   100K .......... .......... .......... .......... ..........  0% 80.0M 0s
   150K .......... .......... .......... .......... ..........  1%  103M 0s
   200K .......... .......... .......... .......... ..........  1% 93.6M 0s
   250K .......... .......... .......... .......... ..........  1%  112M 0s
   300K .......... .......... .......... .......... ..........  2% 86.3M 0s
   350K .......... .......... .......... .......... ..........  

  5200K .......... .......... .......... .......... .......... 31%  870K 1s
  5250K .......... .......... .......... .......... .......... 31%  116M 0s
  5300K .......... .......... .......... .......... .......... 31% 86.2M 0s
  5350K .......... .......... .......... .......... .......... 31% 79.1M 0s
  5400K .......... .......... .......... .......... .......... 32%  103M 0s
  5450K .......... .......... .......... .......... .......... 32% 81.6M 0s
  5500K .......... .......... .......... .......... .......... 32% 73.1M 0s
  5550K .......... .......... .......... .......... .......... 33% 75.7M 0s
  5600K .......... .......... .......... .......... .......... 33% 84.0M 0s
  5650K .......... .......... .......... .......... .......... 33% 98.6M 0s
  5700K .......... .......... .......... .......... .......... 33% 97.9M 0s
  5750K .......... .......... .......... .......... .......... 34%  109M 0s
  5800K .......... .......... .......... .......... .......... 34%  120M 0s
  5850K ....

 11550K .......... .......... .......... .......... .......... 68% 95.7M 0s
 11600K .......... .......... .......... .......... .......... 68%  124M 0s
 11650K .......... .......... .......... .......... .......... 69%  126M 0s
 11700K .......... .......... .......... .......... .......... 69%  140M 0s
 11750K .......... .......... .......... .......... .......... 69%  115M 0s
 11800K .......... .......... .......... .......... .......... 70%  137M 0s
 11850K .......... .......... .......... .......... .......... 70%  104M 0s
 11900K .......... .......... .......... .......... .......... 70% 86.0M 0s
 11950K .......... .......... .......... .......... .......... 70% 86.2M 0s
 12000K .......... .......... .......... .......... .......... 71% 95.3M 0s
 12050K .......... .......... .......... .......... .......... 71%  109M 0s
 12100K .......... .......... .......... .......... .......... 71%  110M 0s
 12150K .......... .......... .......... .......... .......... 72%  116M 0s
 12200K ....

20_newsgroups/
20_newsgroups/alt.atheism/
20_newsgroups/alt.atheism/53366
20_newsgroups/alt.atheism/53367
20_newsgroups/alt.atheism/51247
20_newsgroups/alt.atheism/51248
20_newsgroups/alt.atheism/51249
20_newsgroups/alt.atheism/51250
20_newsgroups/alt.atheism/51251
20_newsgroups/alt.atheism/51252
20_newsgroups/alt.atheism/51253
20_newsgroups/alt.atheism/51254
20_newsgroups/alt.atheism/51255
20_newsgroups/alt.atheism/53070
20_newsgroups/alt.atheism/53071
20_newsgroups/alt.atheism/53072
20_newsgroups/alt.atheism/53090
20_newsgroups/alt.atheism/53134
20_newsgroups/alt.atheism/53135
20_newsgroups/alt.atheism/53136
20_newsgroups/alt.atheism/53137
20_newsgroups/alt.atheism/53138
20_newsgroups/alt.atheism/53139
20_newsgroups/alt.atheism/53140
20_newsgroups/alt.atheism/53141
20_newsgroups/alt.atheism/53142
20_newsgroups/alt.atheism/54125
20_newsgroups/alt.atheism/53213
20_newsgroups/alt.atheism/53214
20_newsgroups/alt.atheism/53215
20_newsgroups/alt.atheism/53216
20_newsgroups/alt.atheism/5321

20_newsgroups/alt.atheism/53584
20_newsgroups/alt.atheism/53585
20_newsgroups/alt.atheism/53586
20_newsgroups/alt.atheism/53587
20_newsgroups/alt.atheism/53588
20_newsgroups/alt.atheism/53589
20_newsgroups/alt.atheism/53590
20_newsgroups/alt.atheism/53591
20_newsgroups/alt.atheism/53592
20_newsgroups/alt.atheism/53593
20_newsgroups/alt.atheism/53594
20_newsgroups/alt.atheism/51167
20_newsgroups/alt.atheism/53595
20_newsgroups/alt.atheism/53596
20_newsgroups/alt.atheism/53597
20_newsgroups/alt.atheism/53598
20_newsgroups/alt.atheism/53599
20_newsgroups/alt.atheism/53600
20_newsgroups/alt.atheism/53601
20_newsgroups/alt.atheism/53602
20_newsgroups/alt.atheism/53603
20_newsgroups/alt.atheism/53604
20_newsgroups/alt.atheism/53605
20_newsgroups/alt.atheism/53606
20_newsgroups/alt.atheism/51275
20_newsgroups/alt.atheism/53607
20_newsgroups/alt.atheism/53609
20_newsgroups/alt.atheism/53610
20_newsgroups/alt.atheism/51276
20_newsgroups/alt.atheism/51119
20_newsgroups/alt.atheism/51120
20_newsg

...

20_newsgroups/talk.religion.misc/84067
20_newsgroups/talk.religion.misc/84068
20_newsgroups/talk.religion.misc/84069
20_newsgroups/talk.religion.misc/84070
20_newsgroups/talk.religion.misc/84071
20_newsgroups/talk.religion.misc/84072
20_newsgroups/talk.religion.misc/84073
20_newsgroups/talk.religion.misc/84074
20_newsgroups/talk.religion.misc/84075
20_newsgroups/talk.religion.misc/84076
20_newsgroups/talk.religion.misc/84077
20_newsgroups/talk.religion.misc/84078
20_newsgroups/talk.religion.misc/84079
20_newsgroups/talk.religion.misc/84080
20_newsgroups/talk.religion.misc/84081
20_newsgroups/talk.religion.misc/84082
20_newsgroups/talk.religion.misc/84083
20_newsgroups/talk.religion.misc/84084
20_newsgroups/talk.religion.misc/84085
20_newsgroups/talk.religion.misc/84086
20_newsgroups/talk.religion.misc/84087
20_newsgroups/talk.religion.misc/84175
20_newsgroups/talk.religion.misc/84088
20_newsgroups/talk.religion.misc/84090
20_newsgroups/talk.religion.misc/84091
20_newsgroups/talk.religi

20_newsgroups/talk.religion.misc/84357
20_newsgroups/talk.religion.misc/84358
20_newsgroups/talk.religion.misc/84359
20_newsgroups/talk.religion.misc/84360
rm: cannot remove '20news-19997*': No such file or directory
directories in 20 newsgroup dataset


listing = 


"alt.atheism
"


alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


We will study the full dataset, but if you want to run things a bit faster you can sample only a fraction of the dataset.  The embeddings and clusters will be different, however.

In [3]:
val corpusFraction = 1
val seed = 0L
val corpusFull: RDD[String] = sc.wholeTextFiles("20_newsgroups/*/*").map(_._2)

val corpus = if (corpusFraction == 1.0){corpusFull} else{
               corpusFull.sample(fraction = corpusFraction, withReplacement = false, seed = seed)}

corpusFraction = 1
seed = 0
corpusFull = MapPartitionsRDD[2] at map at <console>:49
corpus = MapPartitionsRDD[2] at map at <console>:49


MapPartitionsRDD[2] at map at <console>:49

The first document of the corpus if from the `talk.politics.misc directory`, with the filename 179112 

In [4]:
corpus.take(1)

Array("Xref: cantaloupe.srv.cs.cmu.edu misc.legal:61051 talk.politics.misc:179112
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!optilink!cramer
From: cramer@optilink.COM (Clayton Cramer)
Newsgroups: misc.legal,talk.politics.misc
Subject: Re: WACO: Clinton press conference, part 1
Message-ID: <15577@optilink.COM>
Date: 23 Apr 93 21:47:27 GMT
References: <1993Apr21.021301.25113@r-node.hub.org> <93Apr22.234553edt.47633@neat.cs.toronto.edu>
Organization: Optilink Corporation, Petaluma, CA
Lines: 10
In article <93Apr22.234553edt.47633@neat.cs.toronto.edu>, quoctp@cs.toronto.edu (Quoc Tuan Pham) writes:
> Did anyone notice that Clinton was smiling and making jokes just before
...


[Xref: cantaloupe.srv.cs.cmu.edu misc.legal:61051 talk.politics.misc:179112
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!optilink!cramer
From: cramer@optilink.COM (Clayton Cramer)
Newsgroups: misc.legal,talk.politics.misc
Subject: Re: WACO: Clinton press conference, part 1
Message-ID: <15577@optilink.COM>
Date: 23 Apr 93 21:47:27 GMT
References: <1993Apr21.021301.25113@r-node.hub.org> <93Apr22.234553edt.47633@neat.cs.toronto.edu>
Organization: Optilink Corporation, Petaluma, CA
Lines: 10

In article <93Apr22.234553edt.47633@neat.cs.toronto.edu>, quoctp@cs.toronto.edu (Quoc Tuan Pham) writes:
> Did anyone notice that Clinton was smiling and making jokes just before
> this press conference? Considering the number of people killed, this 
> seems very inappropriate to me.

Why?  He, Reno, and the FBI got what they wanted -- a reminder of
who is the boss in America -- the thugs who work for the go

In [5]:
//corpus.persist()

Name: Syntax Error.
Message: 
StackTrace: 

# Data Preparation

First we filter out the small words (<3 letters), and then we convert the corpus into a DataFrame, which is what the ML library uses.

In [6]:
val documentDF = corpus.map(_.toLowerCase.split("\\s")).
                        map(_.filter(_.length > 3).filter(_.forall(java.lang.Character.isLetter))).
                        toDF("text")

documentDF = [text: array<string>]


[text: array<string>]

Here is what the first article looks like after preparation.

In [7]:
documentDF.select("text").show(1,false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                           |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[clinton, press, part, optilink, article, tuan, anyone, notice, that, clinton, sm

## Word2Vec Training

The training of the Word2Vec model is more compute intensive than the actual clustering.  This is because the embedding uses a neural network.  For now, we will consider this as a mapping between groups of words and a vector.  A brief but helpful discussion of Word2Vec can be found at  https://spark.apache.org/docs/2.3.0/mllib-feature-extraction.html#word2vec, and many online resources are available.

Training should take around 10 minutes.

Defaults
- min count = 5
    - The minimum number of times a word must appear in the corpus to be counted as vocabulary
- vector size = 100
    - The dimension of the output vector. 
- Window Size is 5
    - The number of words in a window used for input training.  

In [8]:
val t0TermCount = System.nanoTime
val word2Vec = new Word2Vec().
    setInputCol("text").
    setOutputCol("result").
    setVectorSize(100).
    setMinCount(10)
val model = word2Vec.fit(documentDF)
val dtTermCount = round(System.nanoTime - t0TermCount)
println("time to generate word2vec: " + dtTermCount + "seconds")

time to generate word2vec: 680.152seconds


t0TermCount = 3226194910410768
word2Vec = w2v_9b6a30b49a60
model = w2v_9b6a30b49a60
dtTermCount = 680.152


680.152

The application of the Word2Vec algorithm has the following syntax in the ML Pipeline API

In [9]:
val result = model.transform(documentDF)

result = [text: array<string>, result: vector]


[text: array<string>, result: vector]

In [10]:
result.printSchema

root
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- result: vector (nullable = true)



The left column is the processed article and the right column is the vector representation of the collection of words.  While the input can have many words (using a "one hot encoding" style of input), the right column always has the same number of features (100 in this case).

In [11]:
result.show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|[clinton, press, ...|[-0.0502035705813...|
|[engineer, cowboy...|[-0.0190240844676...|
|[wiretapping, ini...|[-0.0232437958475...|
|[tieing, abortion...|[-0.0046298521373...|
|[expansion, mcgil...|[0.04100256933167...|
|[welcome, police,...|[0.00462032424805...|
|[role, national, ...|[0.02762026362249...|
|[concentrate, chi...|[-0.0356977651354...|
|[welcome, police,...|[-0.0049290160980...|
|[holocaust, memor...|[-0.0448481236408...|
|[real, source, tr...|[-0.0652494278806...|
|[study, percentag...|[0.00148526637805...|
|[abortion, privat...|[-0.0511604974162...|
|[earth, also, cel...|[-0.0254515306928...|
|[limiting, govt, ...|[-0.0213143074118...|
|[study, percentag...|[0.00635822871866...|
|[good, neighbor, ...|[-0.0191137050073...|
|[proof, viability...|[-0.0847958023401...|
|[proof, viability...|[-8.9741921466257...|
|[race, violence, ...|[-0.005263

Here we show the first article only:

In [12]:
result.show(1,false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Now that we have a representation of our words as feature vectors, we can cluster them into "topics", or clusters, and begin to explore the internal structure that the Word2Vec and KMeans algorithms reveal.

In [13]:
//val K = math.floor(math.sqrt(documentDF.count/2)).toInt
val K = 10

K = 10


10

The KMeans algorithm is called in this notebook using the ML Pipeline API, which leverages the DataFrames API and has better performance in general.  

In [14]:
import org.apache.spark.ml.clustering.KMeans

In [15]:
val t0ClusterCount = System.nanoTime
         
val resultW2V = result.withColumnRenamed("result","features")
val kmeans = new KMeans().setK(K).setSeed(1L)
val modelK = kmeans.fit(resultW2V)
val labelsDF = modelK.transform(resultW2V).withColumnRenamed("prediction","labels")


val dtClusterCount = round(System.nanoTime - t0ClusterCount)
println("number of Clusters (k) used: " + K)
println("time to generate cluster: " + dtTermCount + "seconds")

number of Clusters (k) used: 10
time to generate cluster: 680.152seconds


t0ClusterCount = 3226884268615392
resultW2V = [text: array<string>, features: vector]
kmeans = kmeans_e2aaf0f83e0d
modelK = kmeans_e2aaf0f83e0d
labelsDF = [text: array<string>, features: vector ... 1 more field]
dtClusterCount = 337.434


337.434

In [16]:
resultW2V.show()

+--------------------+--------------------+
|                text|            features|
+--------------------+--------------------+
|[clinton, press, ...|[-0.0502035705813...|
|[engineer, cowboy...|[-0.0190240844676...|
|[wiretapping, ini...|[-0.0232437958475...|
|[tieing, abortion...|[-0.0046298521373...|
|[expansion, mcgil...|[0.04100256933167...|
|[welcome, police,...|[0.00462032424805...|
|[role, national, ...|[0.02762026362249...|
|[concentrate, chi...|[-0.0356977651354...|
|[welcome, police,...|[-0.0049290160980...|
|[holocaust, memor...|[-0.0448481236408...|
|[real, source, tr...|[-0.0652494278806...|
|[study, percentag...|[0.00148526637805...|
|[abortion, privat...|[-0.0511604974162...|
|[earth, also, cel...|[-0.0254515306928...|
|[limiting, govt, ...|[-0.0213143074118...|
|[study, percentag...|[0.00635822871866...|
|[good, neighbor, ...|[-0.0191137050073...|
|[proof, viability...|[-0.0847958023401...|
|[proof, viability...|[-8.9741921466257...|
|[race, violence, ...|[-0.005263

We can now display each article, along with its feature vector, and the cluster number that it was assigned to.

In [17]:
labelsDF.show()

+--------------------+--------------------+------+
|                text|            features|labels|
+--------------------+--------------------+------+
|[clinton, press, ...|[-0.0502035705813...|     3|
|[engineer, cowboy...|[-0.0190240844676...|     2|
|[wiretapping, ini...|[-0.0232437958475...|     3|
|[tieing, abortion...|[-0.0046298521373...|     2|
|[expansion, mcgil...|[0.04100256933167...|     2|
|[welcome, police,...|[0.00462032424805...|     3|
|[role, national, ...|[0.02762026362249...|     2|
|[concentrate, chi...|[-0.0356977651354...|     2|
|[welcome, police,...|[-0.0049290160980...|     2|
|[holocaust, memor...|[-0.0448481236408...|     7|
|[real, source, tr...|[-0.0652494278806...|     7|
|[study, percentag...|[0.00148526637805...|     2|
|[abortion, privat...|[-0.0511604974162...|     3|
|[earth, also, cel...|[-0.0254515306928...|     7|
|[limiting, govt, ...|[-0.0213143074118...|     3|
|[study, percentag...|[0.00635822871866...|     2|
|[good, neighbor, ...|[-0.01911

Let's take a look at the first cluster:

In [18]:
labelsDF.filter("labels = 0").show()

+--------------------+--------------------+------+
|                text|            features|labels|
+--------------------+--------------------+------+
|[hitachi, oscilli...|[-0.0834846808010...|     0|
|[atari, processor...|[-0.0448761401124...|     0|
|[voice, synth, ch...|[-0.0995623438308...|     0|
|[oscilloscope, sa...|[-0.1196322398347...|     0|
|[circuit, inquiri...|[-0.0765004193420...|     0|
|[dram, controller...|[-0.0587306980641...|     0|
|[need, help, find...|[-0.0456011835331...|     0|
|[oscilliscopes, s...|[-0.1171103589321...|     0|
|[casio, serial, l...|[-0.0594486877150...|     0|
|[cross, compilers...|[-0.0312366772975...|     0|
|[video, free, uni...|[-0.0092522239512...|     0|
|[circuit, boards,...|[-0.0191166970813...|     0|
|[drive, array, wo...|[-0.0032491331498...|     0|
|[remote, extender...|[-0.0429821987527...|     0|
|[amplifier, into,...|[-0.0051534950528...|     0|
|[power, supply, s...|[0.00759276533217...|     0|
|[telephone, circu...|[-0.03784

That view only shows us the first few words, so let's use the expanded display version:

In [19]:
labelsDF.select("text").filter("labels = 0").show(false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

It sure looks like this particular cluster maps very closely to the contents of the `sci.electronics` directory.  Perhaps this cluster could be a similar topic?  It seems like a reasonable place to start when trying to work through the text. 

Let's take a look at how many articles are in each topic:

In [21]:
labelsDF.groupBy("labels").count().orderBy("labels").show()

+------+-----+
|labels|count|
+------+-----+
|     0| 1278|
|     1| 1222|
|     2| 3568|
|     3| 2330|
|     4|  481|
|     5| 1521|
|     6| 3904|
|     7| 2180|
|     8| 2888|
|     9|  625|
+------+-----+



One very nice result here is that the clusters seem to be somewhat uniformly distributed...each cluster has a significant population, and no one cluster seems to have aggregated too many data points.  It's only the beginning here, but we might be able to identify topics with this approach!

In [22]:
labelsDF.count()

19997

### Explore Work Similarities

In [23]:
val wordVectorsDF = model.getVectors
val vocabSize = wordVectorsDF.count()
println("Vocabulary Size: "+vocabSize)

Vocabulary Size: 14896


wordVectorsDF = [word: string, vector: vector]
vocabSize = 14896


14896

In [24]:
wordVectorsDF.printSchema()

root
 |-- word: string (nullable = true)
 |-- vector: vector (nullable = true)



In [25]:
wordVectorsDF.select("word").show()

+------------+
|        word|
+------------+
|   professed|
|    incident|
|     misfire|
|     serious|
|     acronym|
|      comply|
|      breaks|
|   forgotten|
|    precious|
|       mario|
|   controler|
|inflammatory|
|     sectors|
|   ascension|
|      teresa|
|       boggs|
|       tripe|
|    embedded|
|  respecting|
|      daigle|
+------------+
only showing top 20 rows



In [26]:
model.explainParams()

vectorSize: the dimension of codes after transforming from wo...


inputCol: input column name (current: text)
maxIter: maximum number of iterations (>= 0) (default: 1)
maxSentenceLength: Maximum length (in words) of each sentence in the input data. Any sentence longer than this threshold will be divided into chunks up to the size (> 0) (default: 1000)
minCount: the minimum number of times a token must appear to be included in the word2vec model's vocabulary (>= 0) (default: 5, current: 10)
numPartitions: number of partitions for sentences of words (> 0) (default: 1)
outputCol: output column name (default: w2v_9b6a30b49a60__output, current: result)
seed: random seed (default: -1961189076)
stepSize: Step size to be used for each iteration of optimization (> 0) (default: 0.025)
vectorSize: the dimension of codes after transforming from words (> 0) (default: 100, current: 100)
windowSize: the window size (context words from [-window, window]) (> 0) (default: 5)

Word2Vec is an interesting model in that it can take an arbitrary number of words and return a vector.  The input is one very large vector of $1$s and $0$s, where each row of the vector corresponts to a word in the learned vocabulary.  Thus, an article can have many words that go into the input vector, or it can have only a single word.  The single word inputs are interesting because they can be used to find synonyms.  Let's find some synonyms to some words from our learned model:

In [27]:
val word = "drive"
val topN = 5
val synonymsDF = model.findSynonyms(word, topN)
synonymsDF.show()

+----------+------------------+
|      word|        similarity|
+----------+------------------+
|    floppy|0.7424999475479126|
|    drives|0.7419173121452332|
|controller|0.7130770087242126|
|      scsi|0.7129872441291809|
|      esdi|0.6894485950469971|
+----------+------------------+



word = drive
topN = 5
synonymsDF = [word: string, similarity: double]


[word: string, similarity: double]

In [28]:
val word = "network"
val topN = 5
val synonymsDF = model.findSynonyms(word, topN)
synonymsDF.show()

+---------+------------------+
|     word|        similarity|
+---------+------------------+
|     arpa|0.6670048832893372|
|workplace|  0.61444491147995|
|  pathway|0.5837959051132202|
|   novell|0.5518525838851929|
|   encore|0.5424853563308716|
+---------+------------------+



word = network
topN = 5
synonymsDF = [word: string, similarity: double]


[word: string, similarity: double]

Finally, let's save our work and look at another topic modeling approach...the Latent Dirichlet Allocation!

In [29]:
model.write.overwrite.save("model/word2vec")